# GDELT Demo project showcase

This is the simple presentation of my findings and a bit about the skills involved in getting there. For a more general intro, a review of the skills demonstrated here, and setup instructions, see [README.md](https://github.com/reed9999/gdelt-demo/blob/master/README.md) and children.

**AT PRESENT EVERYTHING ALL FINDINGS ARE EXPLORATORY**

## Skills demonstrated

I've moved this to the main README.md.

## Findings and visualizations

This project is still in early stages, but I'll post small scale findings incrementally. So far I'm just running a few simple descriptives and regressions to get a feel for various aspects of the data.  

**NOTE**: To allow for execution of these demos without placing multiple gigabytes in the repo, I am engineering them to run with sample data when bigger datasets (either my local copy, still a sample but bigger, or on the full dataset) are unavailable. Thus specific metrics may not agree with what I report.

In [1]:
import importlib
import analysis.goldsteinscale_avgtone as ga
importlib.reload(ga)
# This seemed to make things very slow but perhaps it's just browser slowness.
# %reload_ext autoreload
# %autoreload 1

regr = ga.GARegression()
regr.prepare_data()


In [2]:
importlib.reload(ga)
regr.report_descriptives()

<bound method StringMixin.__repr__ of         goldsteinscale     avgtone
count       710.000000  710.000000
unique             NaN         NaN
top                NaN         NaN
freq               NaN         NaN
mean          0.300282   -0.385485
std           4.642020    4.965812
min         -10.000000  -22.206776
25%          -2.000000   -3.936718
50%           1.000000   -0.201449
75%           3.000000    3.589744
max          10.000000   10.074627>
     actiongeo_adm1code actiongeo_countrycode actiongeo_featureid  \
mean                NaN                   NaN                 NaN   
std                 NaN                   NaN                 NaN   

     actiongeo_fullname  actiongeo_lat  actiongeo_long  actiongeo_type  \
mean                NaN      33.037886      -10.774361        2.725989   
std                 NaN      19.675413       76.967443        1.306518   

     actor1code actor1countrycode actor1ethniccode ...  globaleventid  \
mean        NaN               NaN    

### International stability and media tone

Goldstein is a measure of propensity of each *type* of event to promote stability; avgtone is the average tone across all items (mostly media stories, I think) referencing the event. My intuition is that they should be positively correlated, because the media reflects the public's interest in stability. 

#### Descriptives and simple plots
A good place to start, before I do inferential stats, is to get to know the data. I jumped ahead a bit but now am coming back to this.

**COMING NEXT**: Means, stdev, plots -- maybe box plots with interquartile range, maybe histograms 

#### Regression analysis

The most interesting story I've found so far is that the tone of coverage trends downwards for time while controlling for the Goldstein score of each kind of event:

In [4]:
regr.go(plot=False)
regr.print_output()


Dependent variable is avgtone (average tone of documents)

Coefficients on the regression:

    fractiondate: -0.22125647975485963
    goldsteinscale: 0.2118984385977188
MSRE: 11.640945002921537
r2: 0.5516527957142805


Results from running on my local "mini" database (different results from the sample data in this repo):

```
Dependent variable is avgtone (average tone of documents)

Coefficients on the regression:

    fractiondate: -0.20230581030052106
    goldsteinscale: 0.07914705296303393
MSRE: 7.249942180177441
r2: 0.46995787329613636
```

#### Interpretation
Each year is associated with a -0.20 change in average tone, holding constant the theoretical propensity of the events to promote stability (i.e. holding constant the Goldstein score). Each 1.0 point increase in the Goldstein store is associated with a +0.08 change, holding constant the date.

In isolation, all we can tell from the MSRE is that a mean absolute value of the std error would be its square root. In principle avgtone can run from -100 to 100, but in practice it seems to usually be between 0 and 10. So, missing estimates by something around 2 or 3 seems not-great; however, it's a more useful metric to compare to other analyses than in isolation. The r-squared looks pretty nice for just my first attempt at specifying a regression with two independent variables, but I should be cautious about this metric as I run more regressions and risk overfitting.

I have not yet run this on all the data (e.g., with Spark on AWS Elastic MapReduce), just a subset of six data files.

### Research questions
To get interesting answers, I need good questions. This is what makes this project more than just an extended SQL or PySpark tutorial. In this kind of research there's going to be some iteration in RQs, because things you think *a priori* might be interesting turn out not to be, and vice versa. So this is largely a running list of ideas, .

1. **Media tone and stability** - The Rosenstein score is a measure of how each kind of action promotes or erodes stability. The tone score is something about the tone of the media coverage of the event. See above.

1. **Actor affinity by dyad** - Do some actor dyads consistently produce higher scores (Rosenstein or media tone) than others? I'd think this would be a trivial "Yes" because relationships between allies should produce more positive news than those between adversaries. So this is exploring the obvious, but a good sanity check to make sure I'm understanding the nature of this data.

  1. **Asymmetrical relationships** - Just brainstorming. If X threatens Y more than Y threatens X that's also significant and interesting.
  
1. **Longitudinal time series questions** - This dataset is so rich that it could support a lot of this.

1. **Clustering countries or other actors** - If I want to polish up those *k*-means clustering skills, there's probably all kinds of variables I could derive ("number of threats made", "number of overtures to negotiation", "general tone of media coverage", etc.).

1. **Network graph analysis** - I don't actually know much about this topic, but clearly this data would support a lot of inferences about how actors build their networks. 



## A note on data engineering and AWS Elastic MapReduce
It's a bit of a gray area whether learning how to move files around to AWS EMR, get Spark to run my Python scripts without hanging, etc. is properly part of data science or whether it's actually data engineering. I find it valuable to learn these skills even though they don't lead directly to findings above. See the README.md for the skills developed in this subset of the project.
